<a href="https://colab.research.google.com/github/EduardoWS/Calculo-Numerico/blob/main/sistemas_lineares(Gauss_LU_Pivo_LU_banda_Cholesky).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np

def sub_regressiva(U,y):
    n = np.shape(y)[0]
    x = np.zeros(n)

    for i in np.arange(n-1,-1,-1):
        x[i] = (y[i]-U[i,i+1:n].dot(x[i+1:n]))/U[i,i]

    return x

def sub_progressiva(L,b):
    n = np.shape(b)[0]
    z = np.zeros(n)

    for i in np.arange(n):
        z[i] = (b[i]-L[i,0:i].dot(z[0:i]))/L[i,i]

    return z

# Gauss com pivô
def eliminacao_gauss_pivo(A,b):
    n = np.shape(A)[0]
    newA = np.copy(A)
    newb = np.copy(b)

    for k in np.arange(n-1):
        p = np.argmax(np.abs(newA[k:n,k]))
        newA[[k,p],k:n] = newA[[p,k],k:n]
        newb[[k,p]] = newb[[p,k]]

        for i in np.arange(k+1,n):
            m = -newA[i,k]/newA[k,k]
            newA[i,k:n] = newA[i,k:n] + m*newA[k,k:n]
            newb[i] = newb[i] + m*newb[k]

    return newA,newb

# LU com pivô
def fatoraLUpivo(A):
    U = np.copy(A)  
    n = np.shape(U)[0]  
    L = np.eye(n)  
    P = np.copy(L)

    for j in np.arange(n-1):
        p = np.argmax(np.abs(U[j:n,j]))
        P[[j,p],:] = P[[p,j],:]
        U[[j,p],j:n] = U[[p,j],j:n]
        L[[j,p],0:j] = L[[p,j],0:j]

        for i in np.arange(j+1,n):
            L[i,j] = U[i,j]/U[j,j] 
            U[i,j+1:n] = U[i,j+1:n] - L[i,j]*U[j,j+1:n]
            U[i,j] = 0 
    return L,U,P

# Exemplo
A = np.array([[1, 1, 1],  
              [1, 0,10],  
              [0,10, 1]], dtype='double')
b = [0,-48,25]

(newA,newb) = eliminacao_gauss_pivo(A,b)
x_pivo_gauss = sub_regressiva(newA,newb)
print(x_pivo_gauss)

(L,U,P) = fatoraLUpivo(A)
bpivo = P.dot(b)
ypivo = sub_progressiva(L,bpivo)
x_pivo_lu = sub_regressiva(U,ypivo)
print(x_pivo_lu)


[ 2.  3. -5.]
[ 2.  3. -5.]


In [ ]:
#LU banda

import numpy as np
import time

def LU( A ):
    
    n = A.shape[ 0 ]
    
    U = A.copy()
    L = np.eye( n )
    
    for j in range( n - 1 ):
        
        for i in range( j + 1, n ):
                
            L[ i, j ] = U[ i, j ] / U[ j, j ]
            U[ i, j : n ] = U[ i, j : n ] - L[ i, j ] * U[ j, j : n ]
            
    return ( L, U )


def LU_banda( A, p ):
    
    n = A.shape[ 0 ]
    
    U = A.copy()
    L = np.eye( n )
    
    for j in range( n - 1 ):
        
        v = min( n, j + p + 1 )
        for i in range( j + 1, v ):
                
            L[ i, j ] = U[ i, j ] / U[ j, j ]
            U[ i, j : v ] = U[ i, j : v ] - L[ i, j ] * U[ j, j : v ]
            
    return ( L, U )


n = 2000
p = 2

A = np.zeros( ( n, n ) )
for i in range( n ):
    for j in range( max( 0, i - p ), min( n, i + p + 1 ) ):
        A[ i, j ] = np.random.normal()    


start_time = time.time()
( L, U ) = LU( A )
end_time = time.time()
print( end_time - start_time )


start_time = time.time()
( L_banda, U_banda ) = LU_banda( A, p )
end_time = time.time()
print( end_time - start_time )


print( np.linalg.norm( L @ U - A ) )
print( np.linalg.norm( L_banda @ U_banda - A ) )

35.78927946090698
0.039710044860839844
2.594287423993927e-13
2.594287423993927e-13


In [ ]:
from math import sqrt
import numpy as np

def cholesky(A):
    n = np.shape(A)[0]

    L = np.zeros((n,n))

    for i in np.arange(n):
        for j in np.arange(i+1):
            tmp_sum = sum(L[i][k]*L[j][k] 
            for k in np.arange(j))           
            if (i == j):
                L[i][j] = sqrt(A[i][i] - tmp_sum)
            else:
                L[i][j] = ((A[i][j] - tmp_sum)/L[j][j])
    return L
 
A = [[3,2,0], [2,5,1], [0,1,3]]
L = cholesky(A)

print(L)

[[1.73205081 0.         0.        ]
 [1.15470054 1.91485422 0.        ]
 [0.         0.52223297 1.65144565]]
